<a href="https://colab.research.google.com/github/sindhu819/Projects/blob/master/Collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
#!pip install databricks-connect && databricks-connect configure


3688821027448397

1110-064546-piled91
443
databricks-connect test

https://community.cloud.databricks.com/?o=3688821027448397#/setting/clusters/1110-155822-drier250/configuration

In [0]:
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:


import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql import Row

from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

import folium
import html

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Business.csv'
business=pd.read_csv(file)

In [0]:
file1=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Users.csv'
users=pd.read_csv(file1)

In [0]:
business_df = spark.read.csv(file,header=True)

## Data Loading

## Business

In [0]:
# check the dataframe schema

business_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- categories: string (nullable = true)



In [0]:
business_df.show()

+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+---------------+-----+------------+--------------------+
|         business_id|       business_name|             address|     city|state|postal_code|     latitude|      longitude|stars|review_count|          categories|
+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+---------------+-----+------------+--------------------+
|PZ-LZzSlhSe9utkQY...|Carluccio's Tivol...|1775 E Tropicana ...|Las Vegas|   NV|      89119|   36.1000163|   -115.1285285|  4.0|          40|Restaurants, Italian|
|tstimHoMcYbkSC4eB...|Maria's Mexican R...|6055 E Lake Mead ...|Las Vegas|   NV|      89156|   36.1956146|   -115.0405289|  4.5|         184|Mexican, Restaura...|
|kANF0dbeoW34s2vwh...|         Dairy Queen|6125 Spring Mount...|Las Vegas|   NV|      89146|   36.1250311|   -115.2256202|  2.0|          33|Fast Food, Food, ...|
|X8mtoSxY8whtmbDlj...|

In [0]:
business_df.count()

5909

## User

In [0]:
users=spark.read.csv(file1,header=True)

# print the schema of the review dataframe
users.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- fans: string (nullable = true)
 |-- average_stars: string (nullable = true)



## Reviews

In [0]:
review_file=r'/content/drive/My Drive/Project /Data Mining/Dataset/Yelp_Reviews.parquet'
reviews=spark.read.parquet(review_file)

reviews.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)



In [0]:
# show some business records

business_df.select('business_id', 'business_name').show(2)

+--------------------+--------------------+
|         business_id|       business_name|
+--------------------+--------------------+
|PZ-LZzSlhSe9utkQY...|Carluccio's Tivol...|
|tstimHoMcYbkSC4eB...|Maria's Mexican R...|
+--------------------+--------------------+
only showing top 2 rows



In [0]:
# show some user records

users.select('user_id', 'user_name').show(3)

+--------------------+---------+
|             user_id|user_name|
+--------------------+---------+
|-1zQA2f_syMAdA04P...|     Zach|
|-2Pb5d2WBPtbyGT_b...|    mindy|
|-3bsS2i9xqjNnIA1f...|      Kim|
+--------------------+---------+
only showing top 3 rows



In [0]:
# show some review records

reviews.select('user_id', 'business_id', 'stars').show(2)

+--------------------+--------------------+-----+
|             user_id|         business_id|stars|
+--------------------+--------------------+-----+
|3CJUJILq7CLHk_9Or...|I4Nr-MVc26qWr08-S...|  4.0|
|2mxBNBeFrgDszqGS5...|d_L-rfS1vT3JMzgCU...|  5.0|
+--------------------+--------------------+-----+
only showing top 2 rows



#### Spark ALS implementation requires the rating matrix to have the following data types:

In [0]:
ratings_df_schema = StructType(
[StructField('userId', IntegerType()),
 StructField('businessId', IntegerType()),
 StructField('rating', DoubleType())]
)


So, we need to map existing string user_id, and business_id to integer values

In [0]:
# create a new userId column (integer)

user_newid_df = spark.createDataFrame(users.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("user_id", StringType(), True),StructField("userId", IntegerType(), True)]))

user_newid_df.show(3)

+--------------------+------+
|             user_id|userId|
+--------------------+------+
|-1zQA2f_syMAdA04P...|     0|
|-2Pb5d2WBPtbyGT_b...|     1|
|-3bsS2i9xqjNnIA1f...|     2|
+--------------------+------+
only showing top 3 rows



In [0]:
# add the new userId column the user dataframe

a = users.alias("a")
b = user_newid_df.alias("b")
    
user_new_df = a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.userId')])

user_new_df.select('userId','user_id', 'user_name').show(2)

+------+--------------------+---------+
|userId|             user_id|user_name|
+------+--------------------+---------+
|     0|-1zQA2f_syMAdA04P...|     Zach|
|     1|-2Pb5d2WBPtbyGT_b...|    mindy|
+------+--------------------+---------+
only showing top 2 rows



In [0]:
# create a new businessId column (integer)

business_newid_df = spark.createDataFrame(business_df.rdd.map(lambda x: x[0]).zipWithIndex(), \
        StructType([StructField("business_id", StringType(), True),StructField("businessId", IntegerType(), True)]))

business_newid_df.show(5)


+--------------------+----------+
|         business_id|businessId|
+--------------------+----------+
|PZ-LZzSlhSe9utkQY...|         0|
|tstimHoMcYbkSC4eB...|         1|
|kANF0dbeoW34s2vwh...|         2|
|X8mtoSxY8whtmbDlj...|         3|
|bJP4l_BGq2CudEu0m...|         4|
+--------------------+----------+
only showing top 5 rows



In [0]:
# add the new businessId column the business dataframe

a = business_df.alias("a")
b = business_newid_df.alias("b")
    
business_new_df = a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.businessId')])

business_new_df.select('businessId','business_id', 'business_name').show(5)


+----------+--------------------+--------------------+
|businessId|         business_id|       business_name|
+----------+--------------------+--------------------+
|         0|PZ-LZzSlhSe9utkQY...|Carluccio's Tivol...|
|         1|tstimHoMcYbkSC4eB...|Maria's Mexican R...|
|         2|kANF0dbeoW34s2vwh...|         Dairy Queen|
|         3|X8mtoSxY8whtmbDlj...|Imperial Asian Bu...|
|         4|bJP4l_BGq2CudEu0m...|Artisan Fine Dini...|
+----------+--------------------+--------------------+
only showing top 5 rows



In [0]:
#map new userId and businessId in the review dataframe

review_df = reviews.select('user_id', 'business_id', 'stars')


# map the userId
a = review_df.alias("a")
b = user_newid_df.alias("b")
    
review_userId_df = a.join(b, col("a.user_id") == col("b.user_id"), 'inner') \
                     .select([col('a.'+xx) for xx in a.columns] + [col('b.userId')])

# map the businessId
a = review_userId_df.alias("a")
b = business_newid_df.alias("b")

review_userId_businessId_df = a.join(b, col("a.business_id") == col("b.business_id"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns] + [col('b.businessId')])

review_userId_businessId_df.show(10)

+--------------------+--------------------+-----+------+----------+
|             user_id|         business_id|stars|userId|businessId|
+--------------------+--------------------+-----+------+----------+
|0y8ORuC2X1i1UF6SG...|--9e1ONYQuAa-CB_R...|  5.0|    63|      5052|
|3qz_dfwbFwTQeDRzy...|--9e1ONYQuAa-CB_R...|  5.0|   165|      5052|
|9spixZHaqC1JeN1ld...|--9e1ONYQuAa-CB_R...|  2.0|   338|      5052|
|A4GnBOU7ZCTcoQK4e...|--9e1ONYQuAa-CB_R...|  5.0|   349|      5052|
|FtUDjNLhVjlIoeFKm...|--9e1ONYQuAa-CB_R...|  4.0|   522|      5052|
|H0tfWQsGjEBuhXD4W...|--9e1ONYQuAa-CB_R...|  5.0|   561|      5052|
|R0KVWeN9xR-F6j4z5...|--9e1ONYQuAa-CB_R...|  4.0|   910|      5052|
|XZaCs-Gs0SXdZgfG3...|--9e1ONYQuAa-CB_R...|  4.0|  1111|      5052|
|dSGINC_8KV6fxNjeQ...|--9e1ONYQuAa-CB_R...|  5.0|  1326|      5052|
|n9DJHwgYflQ_ms8gB...|--9e1ONYQuAa-CB_R...|  3.0|  1618|      5052|
+--------------------+--------------------+-----+------+----------+
only showing top 10 rows



## Collaborative Fltering

In [0]:
# create the rating dataframe required by the ALS model

rating_df = review_userId_businessId_df.select('userId', 'businessId', review_userId_businessId_df.stars.cast('float').alias('rating'))
rating_df.show(2)
print(' Rating matrx no. of rows :', rating_df.count())
rating_df.printSchema()


+------+----------+------+
|userId|businessId|rating|
+------+----------+------+
|    63|      5052|   5.0|
|   165|      5052|   5.0|
+------+----------+------+
only showing top 2 rows

 Rating matrx no. of rows : 1076129
root
 |-- userId: integer (nullable = true)
 |-- businessId: integer (nullable = true)
 |-- rating: float (nullable = true)



In [0]:
rating_df.limit(5).toPandas()

,userId,businessId,rating
0,63,5052,5.0
1,165,5052,5.0
2,338,5052,2.0
3,349,5052,5.0
4,522,5052,4.0


In [0]:
(train, test) = rating_df.randomSplit([0.8, 0.2], seed=123)


In [0]:
# very lengthy process of Cross Validation

als = ALS(userCol="userId", itemCol="businessId", ratingCol="rating", coldStartStrategy="drop")

param_grid = ParamGridBuilder().addGrid(
    als.rank,
    [10, 15, 20],
).addGrid(
    als.maxIter,
    [10, 15, 20],
).build()

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
)

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5, seed=123)
cv_als_model = cv.fit(train)

# Evaluate the model by computing the RMSE on the test data

als_predictions = cv_als_model.bestModel.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(als_predictions)
print("Root-mean-square error = " + str(rmse))

best_model = cv_als_model.bestModel

#best_rank is 20
best_model.rank

#best_maxIter is 20
(best_model
    ._java_obj     # Get Java object
    .parent()      # Get parent (ALS estimator)
    .getMaxIter()) # Get maxIter

# root mean-square =1.5069

Root-mean-square error = 1.506977878800293


20

In [0]:
# trying higher rank and maxIter need huge resources. try to tune regParam (default value = 0.1)
model_path='drive/My Drive/Dataset.Model'
alsb = ALS(rank=20, maxIter=20, regParam=0.3, userCol="userId", itemCol="businessId", ratingCol="rating", \
               coldStartStrategy="drop", seed=123)
alsb_model = alsb.fit(train)

alsb_predictions = alsb_model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(alsb_predictions)
print("Root-mean-square error = " + str(rmse))

# save the ALS model
alsb_model.write().overwrite().save(model_path + 'als')


Root-mean-square error = 1.3855887240262401


In [0]:
# load a new instance of the saved ALS model
model_path='/content/drive/My Drive/Project /Data Mining/Dataset/Result/Dataset.Model'
alsn_model = ALSModel.load(model_path + 'als')

In [0]:
# generate top 10 business recommendations for each user

userRecoms = alsn_model.recommendForAllUsers(10)
#userRecoms.repartition(3).write.parquet("/content/drive/My Drive/Dataset/Result/collaborative.parquet")

In [0]:
userRecoms.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- businessId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [0]:
# add the column user_id, cache the recommendaton dataframe and show recommedations sample

a = userRecoms.alias("a")
b = user_newid_df.alias("b")
    
all_userRecoms = a.join(b, col("a.userId") == col("b.userId"), 'inner') \
             .select([col('a.'+xx) for xx in a.columns] + [col('b.user_id')])

all_userRecoms.cache()   
all_userRecoms.show(1, truncate=False)

+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|userId|recommendations                                                                                                                                                                      |user_id               |
+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------+
|148   |[[2916, 4.981251], [928, 4.7634716], [5880, 4.713999], [4000, 4.625043], [3550, 4.61011], [5396, 4.6098495], [1892, 4.5809016], [882, 4.561146], [1807, 4.520158], [3256, 4.5194893]]|3IBIIaiOrzVP-LTOLPBqRw|
+------+----------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# show recommendations for soem user id

u_id = 'ZWD8UH1T7QXQr0Eq-mcWYg'

userFlatRec =  spark.createDataFrame(all_userRecoms.filter(col('user_id') == u_id).rdd.flatMap(lambda p: p[1]))
userFlatRec.show()

+----------+------------------+
|businessId|            rating|
+----------+------------------+
|      4000| 5.249392509460449|
|       882|5.2168402671813965|
|       928| 5.196783065795898|
|      1892| 5.143679618835449|
|      4707| 5.118048191070557|
|      3189| 5.077023029327393|
|      5853| 5.013651371002197|
|      1807| 4.991521835327148|
|      3340| 4.989435195922852|
|       418| 4.921987056732178|
+----------+------------------+



In [0]:
# show the recommeded restaurants details

a = business_new_df.alias("a")
b = userFlatRec.alias("b")

user_collab_df = a.join(b, col("a.businessId") == col("b.businessId"), 'inner') \
                         .select([col('a.'+xx) for xx in a.columns] + [col('b.rating')])
    
user_collab_df.select('business_id', 'business_name', 'rating', 'categories').toPandas()

,business_id,business_name,rating,categories
0,AkdyGTQX8gO3J24nPlww_w,The Steakhouse at Treasures,4.921987,"Cocktail Bars, Food, Wineries, Nightlife, Rest..."
1,QCbptr697vAb630rIq9gUQ,Winter In July - A Pop-Up Lunch with Yonaka an...,5.216840,"Chinese, Local Flavor, Japanese, Restaurants"
2,JDv3HcmwK0cPkRa0-2aP4Q,B's Smokehouse Bbq,4.991522,"Food, Barbeque, Street Vendors, Restaurants, F..."
3,XiVHxG2nUSp_bn4dBbs4Cg,Sushi House Manda,5.196783,"Restaurants, Japanese"
4,Mw1Q04VtMm3vG3pBByXRtQ,Taqueria Los Parados,5.077023,"Mexican, Restaurants"
5,YB7WIh2S_-nQFAK58a0MlA,"Tetsuro's Sayonara, Aloha, Going Away Uye At J...",5.249393,"Japanese, Restaurants"
6,CmBnas2zg5JyigmXclc4JQ,Mama Luigi's Pizzeria,5.013651,"Restaurants, Pizza"
7,V4E09shHwjNdiclExeiYLg,Tower Suite Cafe,5.143680,"American (New), Restaurants"
8,FZQ3O3sxICy1HaJLhPbGYA,Bon Breads Baking Company,4.989435,"Bakeries, Restaurants, Food"
9,2ZZmVT4Y8Rk3OIzu18PvFQ,Steve-O's New York Pizza,5.118048,"Restaurants, Pizza"


In [0]:
def showInMap(df):
    
    mp = folium.Map(location=[36.1699, -115.1398], zoom_start=12)

    for i, r in df.toPandas().iterrows():
        folium.Marker(
                    location =[r.latitude, r.longitude], 
                    popup = html.escape(r["business_name"]) + '<br>' + 'Stars: ' + str(r.stars) + '<br>' + 'Reviews: ' + str(r.review_count),    
                    icon = folium.Icon(color='blue')).add_to(mp)

    return mp

In [0]:
all_userRecoms.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- businessId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)
 |-- user_id: string (nullable = true)



In [0]:
userFlatRec =  spark.createDataFrame(all_userRecoms.filter(col('user_id') == '3nDUQBjKyVor5wV0reJChg').rdd.map(lambda x:(x[2],x[1])))


In [0]:
all_userRecoms.createOrReplaceTempView("all_userRecoms")

In [0]:
temp=spark.sql("SELECt user_id , explode(recommendations) as r from all_userRecoms")

In [0]:
all_userRecoms = temp.withColumn("business_id", col("r").getItem('businessId')).withColumn("rating",col("r").getItem('rating')).drop('r')

In [0]:
userFlatRec.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- business_id: integer (nullable = true)
 |-- rating: float (nullable = true)



In [0]:
(all_userRecoms.filter(col('user_id') == '3nDUQBjKyVor5wV0reJChg').rdd.map(lambda x:(x[2],x[1])).flatMap(lambda z: z[1])).take(2)

[Row(businessId=4000, rating=4.658333778381348),
 Row(businessId=882, rating=4.644845485687256)]

In [0]:
userFlatRec.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- businessId: long (nullable = true)
 |    |    |-- rating: double (nullable = true)



In [0]:
userFlatRec =  spark.createDataFrame(all_userRecoms.filter(col('user_id') == '3nDUQBjKyVor5wV0reJChg').rdd.flatMap(lambda p: p[0]))
print(userFlatRec.take(1))
a = userFlatRec.alias("a")
b = business_new_df.alias("b")

return a.join(b, col("a.businessId") == col("b.businessId"), 'inner') \
          .select([col('b.business_id'), col('a.rating'), col('b.business_name'),col('b.categories'),
                                                        col('b.stars'),col('b.review_count'),
                                                        col('b.latitude'),col('b.longitude')]) \
          .orderBy("rating", ascending = False)

In [0]:
def getCollabRecom(u_id):
    

    userFlatRec =  all_userRecoms.filter(col('user_id') == u_id)
    print(userFlatRec.show(1,False))
    a = userFlatRec.alias("a")
    b = business_new_df.alias("b")
    
    return a.join(b, col("a.business_id") == col("b.businessId"), 'inner') \
             .select([col('b.business_id'), col('a.rating'),col('a.user_id'), col('b.business_name'),col('b.categories'),
                                                           col('b.stars'),col('b.review_count'),
                                                           col('b.latitude'),col('b.longitude')]) \
             .orderBy("rating", ascending = False)

In [0]:
u_id= '3nDUQBjKyVor5wV0reJChg' 
result_df = getCollabRecom(u_id)
print(result_df.count())

+----------------------+-----------+--------+
|user_id               |business_id|rating  |
+----------------------+-----------+--------+
|3nDUQBjKyVor5wV0reJChg|4000       |4.658334|
+----------------------+-----------+--------+
only showing top 1 row

None
10


In [0]:
result_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_name: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [0]:
result_df.coalesce(1).write.parquet("/content/drive/My Drive/Project /Data Mining/Dataset/Result/Final/collabrative_result.parquet")